# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [46]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,spearfish,44.4908,-103.8594,20.49,68,0,2.57,US,1724598609
1,1,afaahiti,-17.7500,-149.2833,22.41,80,100,10.47,PF,1724598611
2,2,cabo san lucas,22.8909,-109.9124,30.42,57,40,8.75,MX,1724598612
3,3,howard springs,-12.4970,131.0470,26.95,83,0,1.30,AU,1724598613
4,4,lata,40.1629,-8.3327,26.94,49,0,6.19,PT,1724598614


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [55]:
%%capture --no-display

# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5
)

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [48]:
# Narrow down cities that fit criteria and drop any results with null values
clouds_df = city_data_df.loc[(city_data_df['Cloudiness'] == 0) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21)]

# Drop any rows with null values
clear_df = clouds_df.dropna()

# Display sample data
clear_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,howard springs,-12.4970,131.0470,26.95,83,0,1.30,AU,1724598613
73,73,bethel,41.3712,-73.4140,26.06,69,0,0.00,US,1724598504
79,79,simcoe,42.8334,-80.2997,24.31,50,0,0.76,CA,1724598701
93,93,broome,42.2506,-75.8330,24.81,64,0,0.45,US,1724598531
112,112,fort bragg,35.1390,-79.0060,26.62,64,0,2.24,US,1724598446
119,119,inhambane,-23.8650,35.3833,21.79,62,0,4.43,MZ,1724598746
188,188,sault ste. marie,46.5168,-84.3333,22.16,78,0,2.06,CA,1724598822
304,304,mstsislaw,54.0192,31.7257,26.50,50,0,1.55,BY,1724598954
341,341,fuqing,25.7250,119.3794,24.76,97,0,0.76,CN,1724598902
438,438,yarmouth,41.7057,-70.2286,26.19,74,0,2.57,US,1724599102


### Step 3: Create a new DataFrame called `hotel_df`.

In [49]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clear_df[['City', 'Country','Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,howard springs,AU,-12.4970,131.0470,83,
73,bethel,US,41.3712,-73.4140,69,
79,simcoe,CA,42.8334,-80.2997,50,
93,broome,US,42.2506,-75.8330,64,
112,fort bragg,US,35.1390,-79.0060,64,
119,inhambane,MZ,-23.8650,35.3833,62,
188,sault ste. marie,CA,46.5168,-84.3333,78,
304,mstsislaw,BY,54.0192,31.7257,50,
341,fuqing,CN,25.7250,119.3794,97,
438,yarmouth,US,41.7057,-70.2286,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [54]:

# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key  # Corrected syntax
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    try:
        name_address = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
howard springs - nearest hotel: Rydges Palmerston
bethel - nearest hotel: Hampton Inn Danbury
simcoe - nearest hotel: Comfort Inn
broome - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
inhambane - nearest hotel: Hotel de Inhambane
sault ste. marie - nearest hotel: Holiday Inn
mstsislaw - nearest hotel: Мсціслаў
fuqing - nearest hotel: 天河大酒店
yarmouth - nearest hotel: The Inn At Cape Cod
aixe-sur-vienne - nearest hotel: Le Relais des Tuileries
pochinok - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=ee22c885cf77445c8032dbf7c0af5e0e&filter=circle%3A32.4407%2C54.4068%2C10000&bias=proximity%3A32.4407%2C54.4068


,City,Country,Lat,Lng,Humidity,Hotel Name
3,howard springs,AU,-12.4970,131.0470,83,Rydges Palmerston
73,bethel,US,41.3712,-73.4140,69,Hampton Inn Danbury
79,simcoe,CA,42.8334,-80.2997,50,Comfort Inn
93,broome,US,42.2506,-75.8330,64,No hotel found
112,fort bragg,US,35.1390,-79.0060,64,Airborne Inn Lodging
119,inhambane,MZ,-23.8650,35.3833,62,Hotel de Inhambane
188,sault ste. marie,CA,46.5168,-84.3333,78,Holiday Inn
304,mstsislaw,BY,54.0192,31.7257,50,Мсціслаў
341,fuqing,CN,25.7250,119.3794,97,天河大酒店
438,yarmouth,US,41.7057,-70.2286,74,The Inn At Cape Cod


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [59]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    hover_cols=["Hotel Name", "Country"]
    
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)